# Agent with Tools using LangGraph and ChatAmazonNova

This notebook demonstrates building an agent that can use tools to answer questions.

The agent reasons about which tools to use, executes them, and uses the results to provide accurate answers.

## Key Concepts
- **Tool Binding**: Attach tools to the model
- **Agent Loop**: Call model → Execute tools → Repeat until done
- **Conditional Routing**: Route based on tool call presence

In [ ]:
%env NOVA_API_KEY=<YOUR-API-KEY>
%env NOVA_BASE_URL=https://api.nova.amazon.com/v1/

In [ ]:
%pip install -e .

## Setup and Imports

In [ ]:
from typing import Literal

from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode

from langchain_amazon_nova import ChatAmazonNova

## Define Tools

Create tools that the agent can use.

In [ ]:
@tool
def get_weather(location: str, unit: Literal["celsius", "fahrenheit"] = "fahrenheit") -> str:
    """Get the current weather for a location.
    
    Args:
        location: The city and state, e.g. San Francisco, CA
        unit: The temperature unit to use
    """
    weather_data = {
        "San Francisco, CA": {"temp": 65, "condition": "cloudy"},
        "Seattle, WA": {"temp": 52, "condition": "rainy"},
        "New York, NY": {"temp": 45, "condition": "partly cloudy"},
        "Miami, FL": {"temp": 78, "condition": "sunny"},
    }
    
    data = weather_data.get(location, {"temp": 70, "condition": "unknown"})
    
    if unit == "celsius":
        temp = round((data["temp"] - 32) * 5 / 9, 1)
        unit_str = "°C"
    else:
        temp = data["temp"]
        unit_str = "°F"
    
    return f"Weather in {location}: {data['condition']}, {temp}{unit_str}"


@tool
def calculate(operation: str, a: float, b: float) -> str:
    """Perform a mathematical calculation.
    
    Args:
        operation: The operation (add, subtract, multiply, divide, power)
        a: First number
        b: Second number
    """
    operations = {
        "add": a + b,
        "subtract": a - b,
        "multiply": a * b,
        "divide": a / b if b != 0 else "Error: Division by zero",
        "power": a ** b,
    }
    
    if operation not in operations:
        return f"Error: Unknown operation '{operation}'"
    
    result = operations[operation]
    return f"{a} {operation} {b} = {result}" if not isinstance(result, str) else result


@tool
def search_web(query: str) -> str:
    """Search the web for information.
    
    Args:
        query: The search query
    """
    search_results = {
        "python": "Python is a high-level programming language known for its readability.",
        "nova": "Amazon Nova is a family of foundation models from AWS.",
        "langchain": "LangChain is a framework for developing LLM-powered applications.",
    }
    
    for key, value in search_results.items():
        if key in query.lower():
            return value
    
    return f"Search results for '{query}': Information not found."


tools = [get_weather, calculate, search_web]
print(f"Defined {len(tools)} tools: {[t.name for t in tools]}")

## Define Agent Logic

Create the agent node and routing logic.

In [ ]:
def should_continue(state: MessagesState) -> Literal["tools", "end"]:
    """Determine if we should continue to tools or end."""
    messages = state["messages"]
    last_message = messages[-1]
    
    # If there are tool calls, continue to tools
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    
    return "end"

## Build the Agent Graph

In [ ]:
def create_agent_graph(llm, tools):
    """Create the agent graph."""
    llm_with_tools = llm.bind_tools(tools)
    
    workflow = StateGraph(MessagesState)
    
    # Add nodes
    workflow.add_node("agent", lambda state: {"messages": [llm_with_tools.invoke(state["messages"])]})
    workflow.add_node("tools", ToolNode(tools))
    
    # Set entry point
    workflow.add_edge(START, "agent")
    
    # Add conditional edges
    workflow.add_conditional_edges(
        "agent",
        should_continue,
        {"tools": "tools", "end": END}
    )
    
    # After tools, go back to agent
    workflow.add_edge("tools", "agent")
    
    return workflow.compile()

## Initialize the Agent

In [ ]:
# Initialize model
llm = ChatAmazonNova(
    model="nova-pro-v1",
    temperature=0,  # Use 0 for consistent tool calling
)

# Create agent
agent = create_agent_graph(llm, tools)

print("Agent initialized with tools!")

## Example 1: Weather Query

In [ ]:
query = "What's the weather like in Seattle?"

result = agent.invoke({"messages": [HumanMessage(content=query)]})

print(f"Query: {query}\n")
print(f"Answer: {result['messages'][-1].content}")

## Example 2: Calculation

In [ ]:
query = "What is 25 times 17?"

result = agent.invoke({"messages": [HumanMessage(content=query)]})

print(f"Query: {query}\n")
print(f"Answer: {result['messages'][-1].content}")

## Example 3: Web Search

In [ ]:
query = "What is Amazon Nova?"

result = agent.invoke({"messages": [HumanMessage(content=query)]})

print(f"Query: {query}\n")
print(f"Answer: {result['messages'][-1].content}")

## Example 4: Multi-Tool Query

In [ ]:
query = "What's the weather in Miami and Seattle? Then calculate the temperature difference."

result = agent.invoke({"messages": [HumanMessage(content=query)]})

print(f"Query: {query}\n")
print(f"Answer: {result['messages'][-1].content}")

## Inspect Tool Calls

Let's see what tools were actually called.

In [ ]:
print(f"Total messages in conversation: {len(result['messages'])}\n")

for i, msg in enumerate(result['messages']):
    if msg.type == "human":
        print(f"{i+1}. User: {msg.content}")
    elif msg.type == "ai":
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            print(f"{i+1}. Agent called {len(msg.tool_calls)} tool(s):")
            for tc in msg.tool_calls:
                print(f"   - {tc['name']}: {tc['args']}")
        else:
            print(f"{i+1}. Agent: {msg.content[:100]}...")
    elif msg.type == "tool":
        print(f"{i+1}. Tool result: {msg.content[:100]}...")
    print()

## Try Your Own Query

In [ ]:
# Modify the query below
your_query = "What is 2 to the power of 10?"

result = agent.invoke({"messages": [HumanMessage(content=your_query)]})

print(f"Query: {your_query}\n")
print(f"Answer: {result['messages'][-1].content}")